# <div style="color:white;display:fill;border-radius:5px;background-color:#67A4A4;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:110%;text-align:center">Introduction</p></div>

## <b><span style='color:#4F9898'>Business Goal</span> </b>
To assist a supermarket increase their membership card conversion rate, I will explore different clustering techniques and perform a customer segmentation analysis. Customer segmentation describes the process of identifying similar groups of customers based on commonalities like shopping preferences and purchasing history and allows companies to market to each group more effectively. In this notebook, I will develop K-Means Clustering model to gain a better understanding of the types of customers the company has and identify strategies to increase their membership conversion rate.

<br>

## <b><span style='color:#4F9898'>Data Overview</span> </b>

The [data](https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python) consists of 200 customers with information related to their age, gender, annual income, and spending score. The spending score is a numeric variable ranging from 1 to 100 and was assigned to customers based on behavior parameters and purchasing data. The data set also contains the customer's ID, which will be dropped before beginning the analysis. 

<br>

In [1]:
import os, warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.cluster import hierarchy
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

os.chdir('../input')
cust = pd.read_csv('customer-segmentation-tutorial-in-python/Mall_Customers.csv', sep=',')
cust.rename(columns={"Annual Income (k$)": "Annual Income", "Spending Score (1-100)": "Spending Score"}, inplace=True)
print("There are {:,} observations and {} columns in the data set.".format(cust.shape[0], cust.shape[1]))
print("There are {} missing values in the data.".format(cust.isna().sum().sum()))
cust.head()

There are 200 observations and 5 columns in the data set.
There are 0 missing values in the data.


,CustomerID,Gender,Age,Annual Income,Spending Score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


## <b><span style='color:#4F9898'>Data Summary</span></b>

In [2]:
cust.drop('CustomerID', axis=1, inplace=True)
pd.DataFrame(cust.describe()).style.set_caption("Summary Statistics of Numeric Variables")

,Age,Annual Income,Spending Score
count,200.000000,200.000000,200.000000
mean,38.850000,60.560000,50.200000
std,13.969007,26.264721,25.823522
min,18.000000,15.000000,1.000000
25%,28.750000,41.500000,34.750000
50%,36.000000,61.500000,50.000000
75%,49.000000,78.000000,73.000000
max,70.000000,137.000000,99.000000


In [3]:
cust['Gender'] = ['Women' if i == 'Female' else 'Men' for i in cust.Gender]
pd.DataFrame(cust.select_dtypes('object').describe().T).style.set_caption("Summary Statistics of Categorical Variables")

,count,unique,top,freq
Gender,200,2,Women,112


# <div style="color:white;display:fill;border-radius:5px;background-color:#67A4A4;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:110%;text-align:center">EDA - Exploratory Data Analysis</p></div>

In [4]:
init_notebook_mode(connected=True)
plot_df=cust.copy()
plot_df['Annual Income']=plot_df['Annual Income'].mul(1000)
p1=plot_df.groupby('Gender')['Age'].mean().round(0).astype(int).reset_index()
p2=plot_df.groupby('Gender')['Annual Income'].mean().reset_index()
p3=plot_df.groupby('Gender')['Spending Score'].mean().round(0).astype(int).reset_index()

temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12)))
fig = make_subplots(rows=3, cols=2,
                    subplot_titles=("Distribution of Age<br>by Gender", 
                                    "Customers Average Age",
                                    "Distribution of Income<br>by Gender", 
                                    "Customers Average Income",
                                    "Distribution of Spending<br>by Gender", 
                                    "Customers Average Spending")
                   )

fig.add_trace(go.Histogram(x=plot_df[plot_df.Gender=='Men']['Age'], histnorm='probability density', 
                           marker=dict(opacity=0.7, line=dict(width=1)), 
                           nbinsx=20, name="Men"),
              row=1, col=1)
fig.add_trace(go.Histogram(x=plot_df[plot_df.Gender=='Women']['Age'], histnorm='probability density', 
                           marker=dict(opacity=0.7, line=dict(width=1)),
                           nbinsx=20, name="Women"),
              row=1, col=1)

fig.add_trace(go.Bar(x=p1['Gender'], y=p1['Age'], text=p1['Age'], texttemplate='%{text} years', textposition='outside',
                     marker=dict(opacity=0.8),width=.8,
                     hovertemplate='Average Age Among %{x} = %{y} years<extra></extra>', showlegend=False),
              row=1, col=2)

fig.add_trace(go.Histogram(x=plot_df[plot_df.Gender=='Men']['Annual Income'], histnorm='probability density', 
                           marker=dict(line=dict(width=1)), 
                           opacity=0.7, name="Men", nbinsx=20, showlegend=False),
              row=2, col=1)
fig.add_trace(go.Histogram(x=plot_df[plot_df.Gender=='Women']['Annual Income'], histnorm='probability density', 
                           marker=dict(line=dict(width=1)),
                           opacity=0.7, name="Women", nbinsx=20, showlegend=False),
              row=2, col=1)
fig.add_trace(go.Bar(x=p2['Gender'], y=p2['Annual Income'], text=p2['Annual Income'], 
                     texttemplate='$%{text:,.0f}', textposition='outside',
                     marker=dict(opacity=0.8),width=.8,
                     hovertemplate='Average Income Among %{x} = $%{y}<extra></extra>', showlegend=False),
              row=2, col=2)
fig.add_trace(go.Histogram(x=plot_df[plot_df.Gender=='Men']['Spending Score'], histnorm='probability density', 
                           marker=dict(line=dict(width=1)), 
                           opacity=0.7, name="Men", nbinsx=20, showlegend=False),
              row=3, col=1)
fig.add_trace(go.Histogram(x=plot_df[plot_df.Gender=='Women']['Spending Score'], histnorm='probability density', 
                           marker=dict(line=dict(width=1)),
                           opacity=0.7, name="Women", nbinsx=20, showlegend=False),
              row=3, col=1)
fig.add_trace(go.Bar(x=p3['Gender'], y=p3['Spending Score'], text=p3['Spending Score'], 
                     texttemplate='%{text}', textposition='outside',
                     marker=dict(opacity=0.8),width=.8,
                     hovertemplate='Average Spending Score Among %{x} = %{y}<extra></extra>', showlegend=False),
              row=3, col=2)
fig.update_traces(marker=dict(line=dict(width=1)))
fig.update_layout(template=temp,barmode='overlay', height=1500, width=700,
                  legend=dict(orientation="h", yanchor="bottom", xanchor="right", y=1.03, x=.97),
                  xaxis1_title="Age", yaxis1_title='Probability Density', 
                  xaxis2_title="Gender", yaxis2_title="Age", yaxis2_range=[0,45],
                  xaxis3_title="Annual Income, $", yaxis3_title='Probability Density', 
                  xaxis4_title="Gender", yaxis4_title="Annual Income, $", yaxis4_range=[0,69e3],
                  xaxis5_title="Spending Score", yaxis5_title='Probability Density', 
                  xaxis6_title="Gender", yaxis6_title="Spending Score", yaxis6_range=[0,59]
                 )
fig.show()

# Pairplots
fig = ff.create_scatterplotmatrix(cust, diag='box', index='Gender') 
fig.update_traces(marker=dict(size=9, opacity=0.85, line=dict(width=1)))
fig.update_layout(title="Mall Customer Pair Plots", template=temp, 
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, x=.35),
                  height=900, width=700)
fig.show()

# Correlations
corr=cust.corr()
x = corr.columns.tolist() 
y = corr.index.tolist()
z = corr.values
text = corr.values.round(2)

fig = ff.create_annotated_heatmap(z=z, x=x, y=y, annotation_text=text, colorscale='mint', 
                                  reversescale=True, showscale=True,
                                  hovertemplate="Correlation of %{x} and %{y}= %{z:.3f}")
fig.update_layout(template=temp, title="Mall Customer Correlations", yaxis_tickangle=-30)
fig.show()

## <b><span style='color:#4F9898'>Summary of EDA</span></b>
Overall, the distributions are fairly proportional between men and women. On average, men are slightly older than women and tend to have higher incomes, while women tend to spend more than men.
Based on the correlations and scatterplots, the variables in the data set do not have very strong relationships with each other. There is a weak negative association between `Age` and `Spending Score` of -0.33 and in the scatterplot above, we see that as customers get older, they tend to spend less than younger customers.

# <div style="color:white;display:fill;border-radius:5px;background-color:#67A4A4;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:110%;text-align:center">K-Means Clustering</p></div>
The first clustering technique I will explore is K-Means Clustering. K-Means Clustering is a simple yet powerful clustering method that creates $k$ distinct segments of the data where the variation within the clusters is as small as possible. To find the optimal number of clusters, I will try different values of $k$ and calculate the inertia, or distortion score, for each model. Inertia measures the cluster similarity by computing the total distance between the data points and their closest cluster center. Clusters with similar observations tend to have smaller distances between them and a lower distortion score overall. 

In [5]:
# K-Means Clustering
clust_df = cust.copy()
clust_df['Gender'] = [1 if i == "Women" else 0 for i in clust_df.Gender]

k_means = list()
for clust in range(1,16):
    km = KMeans(n_clusters=clust, init='k-means++', random_state=21).fit(clust_df)
    k_means.append(pd.Series({'Clusters': clust, 
                              'Inertia': km.inertia_,
                              'model': km}))

# Plot results
plot_km = (pd.concat(k_means, axis=1).T
           [['Clusters','Inertia']]
           .set_index('Clusters'))

fig = px.line(plot_km, x=plot_km.index, y='Inertia', markers=True)
fig.add_vline(x=5, line_width=3, line_dash="dash", line_color="darkgrey")
fig.add_annotation(
    xref="x domain",
    yref="y",
    x=.31,
    y=75e3,
    text="Optimal Number of Clusters",
    axref="x domain",
    ayref="y",
    ax=.43,
    ay=12e4,
    arrowhead=2, 
    bordercolor="#585858",
    borderpad=4, 
    bgcolor='white',
    font=dict(size=14)
)
fig.update_traces(line_color='#518C89')
fig.update_layout(template=temp, title="K-Means Clustering Elbow Curve", 
                  xaxis=dict(tickmode = 'linear', showline=True), yaxis=dict(showline=True), width=700)
fig.show()

## <b><span style='color:#4F9898'>Summary</span></b>
The graph above shows the inertia values for each K-Means model with clusters between 1 and 15. The inflection point in the graph occurs at about 5 clusters, where the inertia begins to plateau. This indicates that the optimal number of clusters,  𝑘
  is equal to 5. Below is a plot of the clusters based on their spending score and income.

In [6]:
# K-Means with 5 clusters
km = KMeans(n_clusters=5, random_state=21)
km_pred = km.fit_predict(clust_df)
plot_km=clust_df.copy()
plot_km['K-Means Cluster'] = km_pred
plot_km=plot_km.sort_values(by='K-Means Cluster')
plot_km['K-Means Cluster'] = plot_km['K-Means Cluster'].astype(str)

# Plot of clusters
fig = px.scatter(plot_km, x="Spending Score", y="Annual Income", color="K-Means Cluster", 
                 color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_traces(marker=dict(size=11, opacity=0.75, line=dict(width=1, color='#F7F7F7')))
fig.update_layout(template=temp, title="K-Means Cluster Profiles,<br>Customer Spending vs. Income", 
                  width=700, legend_title='Cluster',
                  xaxis=dict(title='Spending Score', showline=True, zeroline=False), 
                  yaxis=dict(title='Income, $', ticksuffix='k', showline=True))
fig.show()

## <b><span style='color:#4F9898'>Summary of K-Means Clustering</span></b>

The K-Means model segments the data into distinct clusters based on customer's spending and income. Cluster 0 in the center of the graph consists of customers with average spending scores, between 35-61, and incomes between \\$40,000 and $71,000. The two clusters on the left, Clusters 1 and 3, both identify customers with lower spending scores that are below 40 and subdivides the groups according to their income. In contrast, Clusters 2 and 4 consist of customers with higher spending scores, above 61, and are further partitioned based on their income.

In [7]:
# Initializing figure with 3 3D subplots
fig = make_subplots(rows=3, cols=1,
                    vertical_spacing=0.1,
                    specs=[[{'type': 'scatter3d'}],
                           [{'type': 'scatter3d'}], 
                           [{'type': 'scatter3d'}]],
                     subplot_titles=("K-Means Clustering with 5 clusters")
                   )

# Adding clusters to scatterplots 
plot_km['K-Means Cluster'] = plot_km['K-Means Cluster'].astype(int)
plot_km=plot_km.sort_values(by='K-Means Cluster')
for i in range(0,5):
    fig.add_trace(go.Scatter3d(x = plot_km[plot_km['K-Means Cluster'] == i]['Spending Score'],
                               y = plot_km[plot_km['K-Means Cluster'] == i]['Age'],
                               z = plot_km[plot_km['K-Means Cluster'] == i]['Annual Income'],                        
                               mode = 'markers', marker=dict(
                                   size=6,  
                                   color = px.colors.qualitative.Prism[i],
                                   line_width = 1,
                                   line_color='#F7F7F7',
                                   opacity=0.7),
                               name = str('Cluster '+str(i)), legendgroup = 1),
                 row=1, col=1)



fig.update_traces(hovertemplate='Customer Spending Score: %{x}<br>Income: $%{z}<br>Age: %{y}')
fig.update_layout(title="Customer Segments based on Income, Spending, and Age",
                  template=temp, height=1800, legend_tracegroupgap = 500,
                  scene=dict(aspectmode='cube',
                             xaxis = dict(title='Spending Score', 
                                          backgroundcolor="#F3F3F3",
                                          gridcolor="white",
                                          showbackground=True,
                                          zerolinecolor="white",),
                             yaxis = dict(title='Age, in years',
                                          backgroundcolor="#E4E4E4",
                                          gridcolor="white",
                                          showbackground=True,
                                          zerolinecolor="white"),
                             zaxis = dict(title='Income, $', 
                                          ticksuffix='k',
                                          backgroundcolor="#F6F6F6",
                                          gridcolor="white",
                                          showbackground=True,
                                          zerolinecolor="white"))
                  )
fig.show()

# <div style="color:white;display:fill;border-radius:5px;background-color:#67A4A4;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:110%;text-align:center">Conclusion</p></div>

In this analysis, K-Means Clustering to identify distinct customer segments that the company could target depending on their needs.

Based on customer spending habits, the company could target customers for their membership card program using the two clusters found in the K-Means model, where customers have higher spending scores above 60 and would likely be more inclined to becoming a member. 

One of the distinguishing factors between these clusters and the other clusters in the model is that they consist of customers who are under the age of 40, which indicates that the company could aim their membership card marketing program towards customers who are younger than 40 and have higher spending scores. 

To engage customers with lower spending scores, the company could target these groups with more of their popular products and promotions. In the future, to better understand customer preferences, additional data about the frequency and types of purchases made could be used to further customize product offerings to each segment.

<b><p style='color:#4F9898;text-align:center'></p></b>